### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = 'new-project'
groq_api_key = os.environ.get("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It', api_key=groq_api_key)

In [6]:
from langchain_core.messages import SystemMessage,HumanMessage

messages = [
    SystemMessage(content= "Translate the following from English to French"),
    HumanMessage(content= "Hello, how are you?")
]
result = model.invoke(messages)

In [7]:
result

AIMessage(content='Bonjour, comment allez-vous ?  \n', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 27, 'total_tokens': 38, 'completion_time': 0.022, 'prompt_time': 0.002825904, 'queue_time': None, 'total_time': 0.024825904}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-85ef588b-0551-4a83-be5a-9a0ee5251d0c-0', usage_metadata={'input_tokens': 27, 'output_tokens': 11, 'total_tokens': 38})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ?  \n'

#### Using LCEL to chain the components

In [9]:
chain = model | parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nYou can also use these more informal options:\n\n* Salut, ça va ?\n* Coucou, comment vas-tu ?\n'

In [10]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate to following into {language}"

prompt = ChatPromptTemplate(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

In [11]:
result = prompt.invoke(
    {"language": "French",
     "text": "Hello"}
)

In [12]:
result.to_messages()

[SystemMessage(content='Translate to following into French'),
 HumanMessage(content='Hello')]

In [14]:
chain = prompt | model | parser
chain.invoke({"language": "French", "text": "Today is August 10 2024. "})

"Aujourd'hui est le 10 août 2024. \n"